<a href="https://colab.research.google.com/github/akeshri003/bookish-meme/blob/main/05_going_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.makedirs("going_modular", exist_ok=True)

In [2]:
import os
import zipfile

from pathlib import Path

import requests

data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
  print(f"{image_path} directory exists")
else:
  print(f"{image_path} directory doesn't exists, creating one...")
  image_path.mkdir(parents=True, exist_ok=True)

with open(data_path / "pizza_steak_sushi.zip", "wb") as file:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print("Downloading zip file")
  file.write(request.content)

with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as fp:
  print("Extracting dataset")
  fp.extractall(image_path)

os.remove(data_path/"pizza_steak_sushi.zip")

data/pizza_steak_sushi directory doesn't exists, creating one...
Extracting dataset


In [3]:
# Setup train / test dir
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [4]:
from torchvision import datasets, transforms

data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root=train_dir,
                                  transform = data_transform,
                                  target_transform=None)
test_data = datasets.ImageFolder(root=test_dir,
                                transform=data_transform)

print(f"Trai Data:\n{train_data}\nTest Data:{test_data}")

Trai Data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test Data:Dataset ImageFolder
    Number of datapoints: 75
    Root location: data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [5]:
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [6]:
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [7]:
len(train_data), len(test_data)

(225, 75)

In [8]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1, # how many samples per batch?
                              num_workers=1, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7db7047869b0>,
 <torch.utils.data.dataloader.DataLoader at 0x7db704787910>)

In [9]:
img, label = next(iter(train_dataloader))

print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, weight]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 3, 64, 64]) -> [batch_size, color_channels, height, weight]
Label shape: torch.Size([1])


In [10]:
!mkdir going_modular

mkdir: cannot create directory ‘going_modular’: File exists


In [11]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoaders for Image Classification data.
"""
import os

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS
):
  """Creates training snd testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders

  Args:
    train_dir:
    test_dir:
    transform:
    batch_size:
    num_workers:

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """

  train_data = datasets.ImageFolder(root=train_dir,
                                    transform=transform,
                                    target_transform=None)
  test_data = datasets.ImageFolder(root=test_dir,
                                   transform=transform,
                                   target_transform=None)

  # Get class names
  class_names = train_data.classes

  # classes dictionary
  class_dict = train_data.class_to_idx

  # Turn images into data loaders
  train_dataloader = DataLoader(dataset=train_data,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=True,
                                pin_memory=True)
  test_dataloader = DataLoader(dataset=test_data,
                               batch_size=batch_size,
                               num_workers=num_workers,
                               shuffle=True,
                               pin_memory=True)

  return train_dataloader, test_dataloader, class_names

Writing going_modular/data_setup.py


In [12]:
import os
os.getcwd()

'/content'

In [13]:
%%writefile going_modular/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch

from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

Writing going_modular/model_builder.py


In [14]:
import torch

from going_modular import model_builder

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)

model_1 = model_builder.TinyVGG(input_shape=3,
                                hidden_units=10,
                                output_shape=len(class_names)).to(device)
model_1

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [15]:
img_batch, label_batch = next(iter(train_dataloader))

img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

model_1.eval()
with torch.inference_mode():
  pred = model_1(img_single.to(device))

print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim = 1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0208, -0.0020,  0.0095]], device='cuda:0')

Output prediction probabilities:
tensor([[0.3371, 0.3295, 0.3333]], device='cuda:0')

Output prediction label:
tensor([0], device='cuda:0')

Actual label:
0


In [16]:
from tqdm.auto import tqdm
from typing import Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward pass,
  loss calculations, optimizer step).

  Args:
    model: A PyTorch model to be trained
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function
    device: A target device to compute on.

  Returns:
    A tuple of training loss and training accuracy metrics
    in the form (train_loss, train_accuracy).
  """
  # Put the model in train mode.
  model.train()

  # Setup train_loss and train_accuracy values.
  train_loss, train_acc = 0, 0

  # Loop through each example in a batch.
  for batch, (X, y) in tqdm(enumerate(dataloader)):
    # send data to target device.
    X, y = X.to(device), y.to(device)

    # forward pass
    train_logit = model(X)
    pred = torch.softmax(train_logit, dim=1).argmax(dim=1)

    # loss calculation
    loss = loss_fn(train_logit, y)

    # zero grad
    optimizer.zero_grad()

    # backpropagation
    loss.backward()

    # gradient descent
    optimizer.step()

    train_loss += loss.item()
    train_acc += (pred==y).sum().item()/len(pred)

  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc

In [17]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module,
              device:torch.device)->Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
  model: A PyTorch model to be tested.
  dataloader: A DataLoader instance for the model to be tested on.
  loss_fn: A PyTorch loss function to calculate loss on the test data.
  device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
  A tuple of testing loss and testing accuracy metrics.
  In the form (test_loss, test_accuracy). For example:

  (0.0223, 0.8985)
  """
  # Put the model in test mode
  model.eval()

  # setup test loss and test acc
  test_loss, test_acc = 0, 0
  with torch.inference_mode():

    # loop through each example in the batch
    for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # forward pass
      y_logits = model(X)
      test_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

      # loss calculation
      loss = loss_fn(test_pred, y)

      test_loss += loss.item()
      test_acc += (y==test_pred).sum().item()/len(y)

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

  return test_loss, test_acc




In [18]:
from tqdm.auto import tqdm
from typing import Dict, List

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device)->Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
  model: A PyTorch model to be trained and tested.
  train_dataloader: A DataLoader instance for the model to be trained on.
  test_dataloader: A DataLoader instance for the model to be tested on.
  optimizer: A PyTorch optimizer to help minimize the loss function.
  loss_fn: A PyTorch loss function to calculate loss on both datasets.
  epochs: An integer indicating how many epochs to train for.
  device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
  A dictionary of training and testing loss as well as training and
  testing accuracy metrics. Each metric has a value in a list for
  each epoch.
  In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]}
  For example if training for epochs=2:
                {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]}
    """
  results = {"train_loss":[],
              "train_acc":[],
              "test_loss":[],
              "test_acc":[]}

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    print(
        f"Epoch: {epoch+1} | "
        f"train_loss: {train_loss:.4f} | "
        f"test_loss: {test_loss:.4f} | "
        f"test_acc: {test_acc:.4f}"
    )

    results["train_loss"] = train_loss
    results["train_acc"] = train_acc
    results["test_loss"] = test_loss
    results["test_acc"] = test_acc

  return results


In [19]:
%%writefile going_modular/engine.py
"""
Contains functions fo training and testing a PyTorch model
"""

from typing import Tuple, Dict, List

import torch

from tqdm.auto import tqdm

from tqdm.auto import tqdm
from typing import Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward pass,
  loss calculations, optimizer step).

  Args:
    model: A PyTorch model to be trained
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function
    device: A target device to compute on.

  Returns:
    A tuple of training loss and training accuracy metrics
    in the form (train_loss, train_accuracy).
  """
  # Put the model in train mode.
  model.train()

  # Setup train_loss and train_accuracy values.
  train_loss, train_acc = 0, 0

  # Loop through each example in a batch.
  for batch, (X, y) in tqdm(enumerate(dataloader)):
    # send data to target device.
    X, y = X.to(device), y.to(device)

    # forward pass
    train_logit = model(X)
    pred = torch.softmax(train_logit, dim=1).argmax(dim=1)

    # loss calculation
    loss = loss_fn(train_logit, y)

    # zero grad
    optimizer.zero_grad()

    # backpropagation
    loss.backward()

    # gradient descent
    optimizer.step()

    train_loss += loss.item()
    train_acc += (pred==y).sum().item()/len(pred)

  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn:torch.nn.Module,
              device:torch.device)->Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
  model: A PyTorch model to be tested.
  dataloader: A DataLoader instance for the model to be tested on.
  loss_fn: A PyTorch loss function to calculate loss on the test data.
  device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
  A tuple of testing loss and testing accuracy metrics.
  In the form (test_loss, test_accuracy). For example:

  (0.0223, 0.8985)
  """
  # Put the model in test mode
  model.eval()

  # setup test loss and test acc
  test_loss, test_acc = 0, 0
  with torch.inference_mode():

    # loop through each example in the batch
    for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # forward pass
      y_logits = model(X)
      test_pred = torch.softmax(y_logits, dim=1).argmax(dim=1)

      # loss calculation
      loss = loss_fn(test_pred, y)

      test_loss += loss.item()
      test_acc += (y==test_pred).sum().item()/len(y)

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

  return test_loss, test_acc

from tqdm.auto import tqdm
from typing import Dict, List

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs: int,
          device: torch.device)->Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
  model: A PyTorch model to be trained and tested.
  train_dataloader: A DataLoader instance for the model to be trained on.
  test_dataloader: A DataLoader instance for the model to be tested on.
  optimizer: A PyTorch optimizer to help minimize the loss function.
  loss_fn: A PyTorch loss function to calculate loss on both datasets.
  epochs: An integer indicating how many epochs to train for.
  device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
  A dictionary of training and testing loss as well as training and
  testing accuracy metrics. Each metric has a value in a list for
  each epoch.
  In the form: {train_loss: [...],
                train_acc: [...],
                test_loss: [...],
                test_acc: [...]}
  For example if training for epochs=2:
                {train_loss: [2.0616, 1.0537],
                train_acc: [0.3945, 0.3945],
                test_loss: [1.2641, 1.5706],
                test_acc: [0.3400, 0.2973]}
    """
  results = {"train_loss":[],
              "train_acc":[],
              "test_loss":[],
              "test_acc":[]}

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    print(
        f"Epoch: {epoch+1} | "
        f"train_loss: {train_loss:.4f} | "
        f"test_loss: {test_loss:.4f} | "
        f"test_acc: {test_acc:.4f}"
    )

    results["train_loss"] = train_loss
    results["train_acc"] = train_acc
    results["test_loss"] = test_loss
    results["test_acc"] = test_acc

  return results

Writing going_modular/engine.py


In [20]:
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
  model: A target PyTorch model to save.
  target_dir: A directory for saving the model to.
  model_name: A filename for the saved model. Should include
    either ".pth" or ".pt" as the file extension.

  Example usage:
  save_model(model=model_0,
              target_dir="models",
              model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directrory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model name should end with .pth or .pt"
  model_save_path = target_dir_path / model_name

  # Save the model_state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

In [21]:
%%writefile going_modular/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
from pathlib import Path

import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
  model: A target PyTorch model to save.
  target_dir: A directory for saving the model to.
  model_name: A filename for the saved model. Should include
    either ".pth" or ".pt" as the file extension.

  Example usage:
  save_model(model=model_0,
              target_dir="models",
              model_name="05_going_modular_tingvgg_model.pth")
  """

  # Create target dir path
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pth'mor '.pt'"
  model_save_path = target_dir_path / model_name

  # Saving the model
  print(f"[INFO] Saving the model to {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)


Writing going_modular/utils.py


In [22]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os

import torch

from torchvision import transforms

import data_setup, engine, model_builder, utils


# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Writing going_modular/train.py


In [23]:
!python going_modular/train.py

  0% 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.02it/s]
8it [00:01,  6.50it/s]
  0% 0/5 [00:01<?, ?it/s]
Traceback (most recent call last):
  File "/content/going_modular/train.py", line 54, in <module>
    engine.train(model=model,
  File "/content/going_modular/engine.py", line 171, in train
    test_loss, test_acc = test_step(model=model,
  File "/content/going_modular/engine.py", line 109, in test_step
    loss = loss_fn(test_pred, y)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py", line 1293, in forward
    return F.cross_entropy(
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py", line 3479, in cross_entropy
    return torch._C._n